In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import netCDF4
import os
from datetime import timedelta
from itertools import chain
import sys
_HOME_ = os.path.expanduser('~') + "/PHD"
from pandas.tseries.offsets import MonthEnd
import xarray as xr

In [2]:
_NETCDF_FILENAME_ = _HOME_ + "/ISPRA_20152017_Analysis/Clustering/"


In [3]:
xr_temp = xr.open_dataset(_NETCDF_FILENAME_ + "temp.nc").isel(depth =  0).drop_vars('depth')
xr_net_prim_prod = xr.open_dataset(_NETCDF_FILENAME_ + "net_prim_prod.nc").isel(depth =  0).drop_vars('depth').interp_like(xr_temp, method = "nearest")
xr_mld = xr.open_dataset(_NETCDF_FILENAME_ + "mld.nc").interp_like(xr_temp, method = "nearest")
xr_dic = xr.open_dataset(_NETCDF_FILENAME_ + "dic.nc").isel(depth =  0).drop_vars('depth').interp_like(xr_temp, method = "nearest")
xr_sal = xr.open_dataset(_NETCDF_FILENAME_ + "/sal.nc").isel(depth =  0).drop_vars('depth').interp_like(xr_temp, method = "nearest")
xr_o2 = xr.open_dataset(_NETCDF_FILENAME_ + "/o2.nc").isel(depth =  0).drop_vars('depth').interp_like(xr_temp, method = "nearest")
xr_ph = xr.open_dataset(_NETCDF_FILENAME_ + "/ph.nc").isel(depth =  0).drop_vars('depth').interp_like(xr_temp, method = "nearest")
xr_nh4 = xr.open_dataset(_NETCDF_FILENAME_ + "/nh4.nc").isel(depth =  0).drop_vars('depth').interp_like(xr_temp, method = "nearest")
xr_chl_phyc = xr.open_dataset(_NETCDF_FILENAME_ + "/chl_phyc.nc").isel(depth =  0).drop_vars('depth').interp_like(xr_temp, method = "nearest")
xr_no3_po4 = xr.open_dataset(_NETCDF_FILENAME_ + "no3_po4.nc").isel(depth =  0).drop_vars('depth').interp_like(xr_temp, method = "nearest")

In [4]:
xr_vars = xr.merge([xr_temp, 
                    xr_net_prim_prod, 
                    xr_mld, 
                    xr_dic, 
                    xr_sal, 
                    xr_o2, 
                    xr_ph, 
                    xr_nh4, 
                    xr_chl_phyc, 
                    xr_no3_po4
                    ]).rename({'latitude': 'Latitude', 'longitude': 'Longitude', "time" : "Date", "thetao" : "temp", "mlotst" : "mld", "dissic" : "dic", "so" : "sal"})

In [5]:
xr_vars["Date"] = pd.to_datetime(xr_vars["Date"].values) + pd.offsets.MonthEnd(0)
xr_vars = xr_vars.interpolate_na(dim = "Longitude", method="nearest")

In [6]:
id_dates = pd.read_csv(_HOME_ + "/ISPRA_20152017_Analysis/sites_taxa_matrix.csv", usecols = ["Date", "id"])

In [7]:
stations = pd.merge(id_dates, pd.read_csv(_HOME_ + "/ISPRA_20152017_Analysis/Stations_info.csv"), on = "id", how = "left")

In [8]:
Longitude = xr.DataArray(stations["Longitude"].to_numpy(), dims="z")
Latitude = xr.DataArray(stations["Latitude"].to_numpy(), dims="z")
dates = xr.DataArray(stations["Date"].to_numpy(), dims="z")

In [9]:
xr_slice = xr_vars.sel(Latitude = Latitude, Longitude = Longitude, Date = dates, method = "nearest")

In [18]:
xr_slice["id"] = xr.DataArray(stations["id"].to_numpy(), dims="z")
xr_slice["Closest_coast	"] = xr.DataArray(stations["Closest_coast"].to_numpy(), dims="z")
xr_slice["Region"] =  xr.DataArray(stations["Region"].to_numpy(), dims="z")

In [20]:
xr_slice.to_dataframe().to_csv(_HOME_ + "/ISPRA_20152017_Analysis/Clustering/Cop_variables_on_sampling_sites.csv", index = False)